In [ ]:
import numpy as np
import numpy.linalg
import numpy.matlib
import time
import os
import cv2
import numba
import math
import joblib
import threading
from queue import Queue

In [ ]:
################################  HONV_LLC  #####################################

In [ ]:
import HONV_LLC_Library as LLC

In [ ]:
codebook = joblib.load('kmeans_HONV_512.pkl')
centers = codebook.cluster_centers_
print(centers.shape)

In [ ]:
################################  Read Images  #####################################

In [ ]:
# this function is for image reading,the input is directory name
def read_directory(directory_name):
    array_of_img = [] # this if for store all of the image data
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #print(filename) #just for test
        #img is used to store the image data 
        img = cv2.imread(directory_name + "/" + filename, cv2.IMREAD_ANYDEPTH)
        array_of_img.append(img)
    return array_of_img

In [ ]:
array_of_bowl = read_directory("training_imgs/Depth_imgs/bowl")
print(len(array_of_bowl))
array_of_cap = read_directory("training_imgs/Depth_imgs/cap")
print(len(array_of_cap))
array_of_flashlight = read_directory("training_imgs/Depth_imgs/flashlight")
print(len(array_of_flashlight))

In [ ]:
################################  Threading  #####################################

In [ ]:
def Image_Encoding(imgs, centers, q):
    l = len(imgs)
    for i in range(l):
        img_feature = LLC.Img_code(imgs[i], centers) #HONV_LLC
        q.put(img_feature)

In [ ]:
#here we create 3 threading for acceleration of encoding for multi class, you can choose to encode 1 class at on time without using it. 
def multithreading(centers, class_1_imgs, class_2_imgs, class_3_imgs):
    q_class_1 =Queue() 
    q_class_2 =Queue()
    q_class_3 =Queue()
    
    threads = []
    
    class_1 = threading.Thread(target=Image_Encoding,args=(class_1_imgs,centers,q_class_1))
    class_1.start()
    threads.append(class_1) #category 1
    class_2 = threading.Thread(target=Image_Encoding,args=(class_2_imgs,centers,q_class_2))
    class_2.start()
    threads.append(class_2) #category 2
    class_3 = threading.Thread(target=Image_Encoding,args=(class_3_imgs,centers,q_class_3))
    class_3.start()
    threads.append(class_3) #category 3
    
    
    for thread in threads:
        thread.join()
    
    class_1_codes = []
    for i in range(len(class_1_imgs)):
        class_1_codes.append(q_class_1.get())        
    class_2_codes = []
    for i in range(len(class_2_imgs)):
        class_2_codes.append(q_class_2.get())        
    class_3_codes = []
    for i in range(len(class_3_imgs)):
        class_3_codes.append(q_class_3.get())
    
    return class_1_codes, class_2_codes, class_3_codes

In [ ]:
bowl_results, cap_results, flashlight_results = multithreading(centers, array_of_bowl, array_of_cap, array_of_flashlight)

bowl_results = np.array(bowl_results)
cap_results = np.array(cap_results)
flashlight_results = np.array(flashlight_results)

print(bowl_results.shape)
print(cap_results.shape)
print(flashlight_results.shape)

print(type(flashlight_results))

In [ ]:
np.savetxt("bowl_HONV_LLC512.csv", bowl_results, delimiter=",")
np.savetxt("cap_HONV_LLC512.csv", cap_results, delimiter=",")
np.savetxt("flashlight_HONV_LLC512.csv", flashlight_results, delimiter=",")